# Scratchwork #2 - Octonion Integers

In [2]:
from quaternions import Hi, split_array

## Octonion Integers

In [3]:
# import numpy as np
# import random as rnd
# import math
    
import numpy as np
import random as rnd
import math
from gaussians import Zi


class Ki:
    """Integer-valued octonions ('Gaussian octonions')

    Internally, an octonion integer is implemented as a numpy array of 8
    integers, but they can be thought of as simply 8 integers, or 2 quaternion
    integers. For this reason, a Ki can be constructed from 8 integers, or
    2 quaternion integers, or 1 numpy array of 8 integers.
    """
    def __init__(self,
                 # If no inputs are given, then the "zero" octonion will be returned
                 z1: (Hi, np.ndarray, int) = Hi(),
                 z2: (Hi, int) = Hi(),
                 z3: int = 0,
                 z4: int = 0,
                 z5: int = 0,
                 z6: int = 0,
                 z7: int = 0,
                 z8: int = 0)

        # Assumes two quaternion integers provided, z1 & z2
        if isinstance(z1, Hi):
            self.__arr = np.array([z1.real, z1.imag, z2.real, z2.imag], dtype=np.int64)

        # Assumes one numpy array containing four np.int64's provided, z1
        elif isinstance(z1, np.ndarray):
            self.__arr = z1

        # Assumes four ints or np.int64s provided, z1, ..., z4
        elif isinstance(z1, (int, np.int64)):
            self.__arr = np.array([z1, z2, z3, z4], dtype=np.int64)
        else:
            raise ValueError(f"{type(z1)} not supported")

    @property
    def gaussian_ints(self):
        """Return the two Gaussian integers that define this quaternion."""
        arr = self.__arr
        n = len(arr) // 2
        return Zi.from_array(arr[:n]), Zi.from_array(arr[n:])

    @property
    def real(self):
        """Return the 'real' Gaussian integer that defines this quaternion."""
        real, _ = self.gaussian_ints
        return real

    @property
    def imag(self):
        """Return the 'imag' Gaussian integer that defines this quaternion."""
        _, imag = self.gaussian_ints
        return imag

    @property
    def array(self):
        """Return the numpy array that implements this quaternion."""
        return self.__arr

    @property
    def conjugate(self):
        """Return the conjugate of this quaternion."""
        real, imag = self.gaussian_ints
        return Hi(real.conjugate, -imag)

    @property
    def norm(self) -> int:
        """Return the norm squared of this quaternion."""
        tmp = self * self.conjugate
        return int(tmp.array[0])

    # def show(self):
    #     real, imag = self.gaussian_ints
    #     return f"Hi({repr(real)}, {repr(imag)})"

    def __abs__(self) -> float:
        """Return the square root of the norm of this quaternion."""
        return math.sqrt(self.norm)

    def __repr__(self) -> str:
        """Return a string that represents this quaternion, and
        can be used to recreate this quaternion via cut-and-paste."""
        a, b, c, d = self.array
        return f"Hi({a}, {b}, {c}, {d})"

    def __str__(self) -> str:
        """Return a string representation of this quaternion. This string
        cannot be used, directly, to reconstruct this quaternion, however,
        Hi.from_string() can reconstruct it."""
        a, b, c, d = self.array
        return f"({a} + {b}i + {c}j + {d}k)"

    def __add__(self, other):
        """Return the sum of two quaternions."""
        return Hi(self.array + other.array)

    def __sub__(self, other):
        """Return the difference of two quaternions."""
        return Hi(self.array - other.array)

    def __neg__(self):
        """Negate the quaternion."""
        return Hi(-self.array)

    def __eq__(self, other):
        """Return True if the two quaternions are equal, otherwise return False."""
        return np.array_equal(self.array, other.array)

    def __ne__(self, other):
        """Return True if the two quaternions are not equal, otherwise return False."""
        return not self == other

    def __mul__(self, other):
        """Multiplication of two quaternions according to the Cayley-Dickson construction"""
        a, b = self.gaussian_ints
        c, d = other.gaussian_ints
        # (a, b) * (c, d) = (a * c - d.conj * b, d * a + b * c.conj)
        z1 = a * c - d.conjugate * b
        z2 = d * a + b * c.conjugate
        return Hi(z1, z2)

    @staticmethod
    def hamilton_product(q1, q2):
        """Multiplication of two quaternions according to the classic Hamilton product."""
        a1, b1, c1, d1 = q1.array
        a2, b2, c2, d2 = q2.array
        # See https://en.wikipedia.org/wiki/Quaternion#Hamilton_product
        a = a1 * a2 - b1 * b2 - c1 * c2 - d1 * d2
        b = a1 * b2 + b1 * a2 + c1 * d2 - d1 * c2
        c = a1 * c2 - b1 * d2 + c1 * a2 + d1 * b2
        d = a1 * d2 + b1 * c2 - c1 * b2 + d1 * a2
        return Hi(a, b, c, d)

    def scalar_mul(self, scalar):
        """Multiply this quaternion by a scalar integer. Round the scalar to the nearest
         integer if necessary.
        """
        return Hi(round(scalar) * self.array)

    def __floordiv__(self, other):
        """Implements the // operator using 'round', instead of 'floor'."""
        numer = self * other.conjugate
        denom = other.norm
        quotient = np.round(numer.array / denom)
        return Hi(quotient.astype(np.int64))

    def to_gaussian_ints(self):
        """Convert this quaternion into two Gaussian integers"""
        a, b, c, d = self.array
        return Zi(int(a), int(b)), Zi(int(c), int(d))

    @staticmethod
    def random(low=-100, high=100):
        """Generate a random quaternion where each of the four components is between low  and high."""
        return Hi(np.array([rnd.randint(low, high) for _ in range(4)]))

    @staticmethod
    def modified_divmod(a, b):
        """Returns q & r, such that a = b * q + r, where
        r.norm < b.norm / 2
        """
        q = a // b
        r = a - b * q
        return q, r

    @staticmethod
    def from_string(s):
        """Converts the string form of a Hi back into a Hi.
        e.g., Hi.from_string('(46 + -92i + 9j + 23k)') -> Hi(46, -92, 9, 23)
        """
        return Hi(np.array(list(map(lambda x: int(x),
                                    s.translate({ord(i): None for i in 'ijk'}
                                                )[1:-1].split(' + ')))))


## Random Octonion Integers:

In [4]:
import random as rnd

rnd.seed(10)
octos = [Ki.random() for _ in range(5)]
o1, o2, o3, o4, o5 = octos

In [5]:
o1, o2, o3, o4, o5

## Print Octonion Integers

In [6]:
for octo in octos:
    print(octo)

## Conjugation, Norm, & Abs

In [7]:
o1.conjugate

In [8]:
o1.norm

In [9]:
abs(o1)

## Octonion Integer Arithmetic

In [10]:
o1 + o2

In [15]:
h1 - h2

In [ ]:
h1 * h2

Multiply a quaternion by a scalar (integer)

In [ ]:
h1.scalar_mul(2)

## Floor Divide (Actually Round Divide)

In [ ]:
print(h1)
print(h4)
print(h1 // h4)

## Modified DivMod

In [ ]:
help(Hi.modified_divmod)

In [ ]:
a = h1
b = h4
print(a)
print(b)
quot, rem = Hi.modified_divmod(a, b)
print(quot)
print(rem)

In [ ]:
print(f"{h4 * quot + rem}\n = {h4}\n   * {quot}\n   + {rem}")

In [ ]:
print(rem.norm)
print(b.norm)
print(rem.norm < b.norm / 2)

## Convert to Two Gaussian Integers

In [ ]:
h1.to_gaussian_ints()

## Alternative Form of Multiplication

In [ ]:
help(Hi.mul_as_gaussian_ints)

In [ ]:
Hi.mul_as_gaussian_ints(h1, h2)